In [12]:
"""
    The slideshare crawler can download any slide from slideshare.com. Just input slide's url and you will
    get a new folder with all slides in jpg format indefault. An anternative PDF format would be genertated if 
    set optional parameter, pdf, to True.

"""



def slideshare_crawler(url, pdf=None):
    import requests
    import shutil
    import os
    from bs4 import BeautifulSoup
    import img2pdf
    import subprocess
    import re
    
    """
        Input:
           url -> https://www.slideshare.net/tw_dsconf/ss-73014464  # url from slideshare.com
           pdf -> convert slide images to a single pdf file if it is true
        Output:
          Defutlt:
              create a new directory containing all slides under current working directory
          PDF mode:
              save all slide into one pdf file under current working directory
    """

    try: 
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'lxml')
       

        slides =[slide['data-full'] for slide in  soup.select('section.slide > img')]          # Get links of slides

        file_name = slides[0].split('com/')[1].split('-')[0]              # Get folder or file name
        os.chdir(os.getcwd())
        os.mkdir(file_name)

        for slide in slides:
            res2 =requests.get(slide, stream=True)
            pic_name = pic_name = '{:03}.jpg'.format(int(re.search(r'-(\d+)-\d+\.jpg', slide).group(1))) # output xxx.jpg
            with open('./' + file_name + '/' + pic_name, 'wb') as f:
                ## decompress the content
                res2.raw.decode_content = True
                shutil.copyfileobj(res2.raw, f)
        
        # Merge slides into one pdf file
        if pdf == True:
            os.chdir('./' + file_name)
            images = os.listdir()
            images.sort()
            filepath = '../' + file_name + '.pdf'
            with open(filepath, 'wb') as wf:
                wf.write(img2pdf.convert(images))

            # Delete folder after creating pdf file
            os.chdir('..')
            subprocess.run(['rm', '-rf', file_name])
            print('Downloaded Successfully!')

    except:
        # Delete folder if process fails
        subprocess.run(['rm', '-rf', file_name])
        print('some errors happen, please try again')
    

In [13]:
slideshare_crawler('https://www.slideshare.net/ElodieAscenci/10-ways-to-spread-the-love-in-the-office', pdf=True)


UnboundLocalError: local variable 'file_name' referenced before assignment